In [ ]:
from tensorflow import keras

# Load the saved model
m1 = keras.models.load_model('C:\\Users\\Santosh\\Desktop\\fyp\\src\\saved_models\\cnn_landmark_model.keras')


In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
def load_image(image_path):
    return cv2.imread(image_path)

In [ ]:
def skeletonize_image(image):
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = pose.process(image_rgb)
    
    skeleton_image = np.zeros_like(image)
    if result.pose_landmarks:
        mp_drawing.draw_landmarks(
            skeleton_image, 
            result.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=5, circle_radius=5),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
        )
    return skeleton_image

In [ ]:
def preprocess_image(image, target_size=(75, 75)):
    img = cv2.resize(image, target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
def predict(preprocessed_image, model):
    pose_map = {0: 'downdog', 1: 'goddess', 2: 'plank', 3: 'tree', 4: 'warrior'}
    predi = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(predi, axis=1)[0]
    predicted_pose_name = pose_map[predicted_class_index]
    return predicted_pose_name

In [ ]:
def image_processing_pipeline(image_path, output_path=None):
    original_image = load_image(image_path)
    skeleton_image = skeletonize_image(original_image)
    
    if output_path:
        cv2.imwrite(output_path, skeleton_image)
    
    preprocessed_image = preprocess_image(skeleton_image)
    prediction = predict(preprocessed_image, m1)
    return prediction

In [ ]:
def process_images_in_folder(input_folder, output_folder, model):
    predictions = []
    true_labels = []
    
    # Map folder names to class indices (ensure these match the model output)
    pose_map = {'downdog': 0, 'goddess': 1, 'plank': 2, 'tree': 3, 'warrior2': 4}
    
    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(input_folder):
        for filename in files:
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(root, filename)
                
                # Extract the true label from the folder name and normalize to lowercase
                true_label = os.path.basename(root).lower()
                
                if true_label not in pose_map:
                    print(f"Skipping {filename} - Unknown label {true_label}")
                    continue  # Skip folders that are not in the pose_map
                
                # Append the true label index
                true_labels.append(pose_map[true_label])
                
                # Create corresponding output folder structure
                relative_path = os.path.relpath(root, input_folder)
                output_subfolder = os.path.join(output_folder, relative_path)
                if not os.path.exists(output_subfolder):
                    os.makedirs(output_subfolder)
                
                # Save the skeletonized image in the corresponding subfolder
                output_image_path = os.path.join(output_subfolder, f"skeletonized_{filename}")
                
                # Process each image and get the predicted class index
                predicted_class_index = image_processing_pipeline(image_path, output_image_path)
                predictions.append(predicted_class_index)
                
                print(f"Processed {filename}: True Pose = {true_label}, Predicted Pose = {predicted_class_index}")
    
    # Check if true_labels or predictions is empty
    if len(true_labels) == 0 or len(predictions) == 0:
        print("No valid labels or predictions found. Please check the folder structure and the model.")
        return None, None, float('nan')
    
    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
    return predictions, true_labels, accuracy

In [ ]:
#use if ram is less

# import os
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# import numpy as np
# from sklearn.metrics import accuracy_score
# import mediapipe as mp
# import cv2
# Initialize MediaPipe Pose.
# mp_pose = mp.solutions.pose
# mp_drawing = mp.solutions.drawing_utils  # Utility to draw landmarks and connections
# pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# # Main input and output folder paths
# input_main_folder = '/kaggle/input/yoga-poses-dataset/DATASET/TRAIN'
# output_main_folder = '/kaggle/working/output_pic_withline/train'

# # Create output main folder if it doesn't exist
# os.makedirs(output_main_folder, exist_ok=True)

# # Loop through each subfolder (pose type)
# for subfolder in os.listdir(input_main_folder):
#     input_folder = os.path.join(input_main_folder, subfolder)
#     output_folder = os.path.join(output_main_folder, subfolder)
    
#     # Check if the subfolder is a directory (pose folder)
#     if os.path.isdir(input_folder):
#         # Create output folder for each pose if it doesn't exist
#         os.makedirs(output_folder, exist_ok=True)

#         # Process each image in the input folder
#         for filename in os.listdir(input_folder):
#             if filename.endswith(('.png', '.jpg', '.jpeg')):
#                 # Read the image
#                 image_path = os.path.join(input_folder, filename)
#                 image = cv2.imread(image_path)
#                 image_height, image_width, _ = image.shape

#                 # Convert the image to RGB
#                 image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#                 # Process the image and find the pose
#                 result = pose.process(image_rgb)

#                 # Create a blank numpy array with zeros
#                 skeleton_image = np.zeros((image_height, image_width, 3), dtype=np.uint8)

#                 # Draw the skeleton on the blank image
#                 if result.pose_landmarks:
#                     mp_drawing.draw_landmarks(
#                         skeleton_image, 
#                         result.pose_landmarks, 
#                         mp_pose.POSE_CONNECTIONS,  # This connects the keypoints to form the skeleton
#                         landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=5, circle_radius=5),
#                         connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
#                     )

#                 # Save the image with the skeleton
#                 output_path = os.path.join(output_folder, filename)
#                 cv2.imwrite(output_path, skeleton_image)

# print("Processing complete.")


# # Directory containing the test folders
# test_dir = '/kaggle/working/output_pic_withline/test'  # Update with your test dataset path
# pose_folders = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']  # Update with your actual folder names

# pose_map = {0: 'downdog', 1: 'goddess', 2: 'plank', 3: 'tree', 4: 'warrior2'}

# # Initialize lists to store true labels and predictions
# true_labels = []
# predictions = []
# predicted_pose_names = []

# # Load and preprocess images
# for label, folder in enumerate(pose_folders):
#     folder_path = os.path.join(test_dir, folder)
#     for img_name in os.listdir(folder_path):
#         img_path = os.path.join(folder_path, img_name)
#         img = load_img(img_path, target_size=(75, 75))  # Same size as your training images
        
#         # Convert the image to a numpy array and normalize it
#         img_array = img_to_array(img)
#         img_array = img_array / 255.0  # Rescale the image
        
#         # Add an additional dimension for batch size (as the model expects a batch of images)
#         img_array = np.expand_dims(img_array, axis=0)
        
#         # Predict the class of the image
#         pred = model.predict(img_array)
        
#         # Get the predicted class index
#         predicted_class_index = np.argmax(pred, axis=1)[0]
        
#         # Append true label and prediction
#         true_labels.append(label)
#         predictions.append(predicted_class_index)
        
#         predicted_pose_name = pose_map[predicted_class_index]
#         predicted_pose_names.append(predicted_pose_name)

# # Calculate accuracy
# accuracy = accuracy_score(true_labels, predictions)
# print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
input_folder_path = 'C:\\Users\\Santosh\\Desktop\\fyp\\YogaPoses_200'
output_folder_path = 'C:\\Users\\Santosh\\Desktop\\fyp\\YogaPoses_200_op'

In [ ]:
predictions, true_labels, accuracy = process_images_in_folder(input_folder_path, output_folder_path, m1)

In [ ]:
if accuracy is not None:
    print(f"Final accuracy: {accuracy * 100:.2f}%")
else:
    print("No accuracy could be calculated due to missing data.")